<a href="https://colab.research.google.com/github/utkar22/CSE508_Winter2023_A1_48/blob/main/Using_Positional_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pickle
from typing import List
import os
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from typing import List
class preprocessor:
  def __init__(self):
    self.value = 1
  
  def read_file(self, path:str, type_of:str) -> str:

    '''reads the contents of a file and converts them into 
    a string of words separated by space.'''
    f = open(path, 'r')
    text = ''
    words = []
    for line in f:
      for word in line.split(" "):
        text += word + ' '
        words.append(word)
    f.close()
    return text if type_of == 'text' else words
  
  def tokenize(self, text:str):
    ''' The function takes in a string and 
    converts it into a list of tokens using nltk'''

    tokens = word_tokenize(text)
    return tokens
  def tokens_to_text(self, tokens:List) -> str:
    ''' converts the tokens into space-separated string'''
    text = ''
    for token in tokens:

      text += '' + token + ' '
    return text

  def lowercase(self, tokens:List):
    ''' This function takes in the list of tokens
     and returns the lowercase version of each token without changing the index positions'''

    for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()
    return tokens

  def remove_stopwords(self, tokens:List) -> List:
    ''' removes any stop words present as tokens in the list of tokens 
    present using nltk's corpus of stopwords'''
    stop_words = set(stopwords.words('english'))
    resultant_tokens = []
    for token in tokens:
      if token not in stop_words:
        resultant_tokens.append(token)
    return resultant_tokens

  def remove_puncts(self, text:str) -> List:
    ''' removes any punctuations present as tokens 
    and returns the resutlant list of tokens'''

    resultant_tokens = []
    text = text.translate(str.maketrans('', '', string.punctuation))
    resultant_tokens = self.tokenize(text)
    return resultant_tokens

  def write_to_file(self, path:str, text:str) ->None:
    '''wrtie the text to the file '''
    open(path, 'w').close()
    f = open(path, 'w')
    f.write(text)
    f.close()

  def make_doc_id_mapping(self, path_to_folder:str)->dict:
    '''make the mapping of doc_id : doc'''
    mapping = {}
    folder = os.fsencode(path_to_folder)
    for doc in os.listdir(folder):
      doc_name = os.fsdecode(doc)
      if doc_name.find('cranfield') == -1:
        continue
      number = (int)(doc_name[-4:])
      mapping[number] = doc_name
    return mapping



In [8]:
class word:
  ''' Contains the {count of documents which contain the word, a map of doc_id to the positions in that doc}'''
  def __init__(self):
    self.value = None
    self.freq = 1
    self.docs = {}

class doc:
  def __init__(self, did):
    self.docId = did
    self.positions = []
dictionary = {}

In [9]:
doc_id_mapping = {}
with open('/content/drive/MyDrive/IR_Assignments/A1/doc_id_mapping.pkl', 'rb') as f:
  doc_id_mapping = pickle.load(f)
print(len(doc_id_mapping))

1400


In [11]:
dictionary = {}
with open('/content/drive/MyDrive/IR_Assignments/A1/positional_index.pkl', 'rb') as f:
  dictionary = pickle.load(f)
print(len(dictionary))

8996


In [12]:
proc = preprocessor()
def preprocess_query(phrase:str)->List:
  phrase_tokens = proc.tokenize(phrase)
  phrase_tokens = proc.lowercase(phrase_tokens)
  phrase_tokens = proc.remove_stopwords(phrase_tokens)
  phrase_tokens = proc.remove_puncts(proc.tokens_to_text(phrase_tokens))
  return phrase_tokens

def intersect_docs(phrase:str, dictionary:dict, cands:set):
  phrase_tokens = preprocess_query(phrase)
  for token in phrase_tokens:
    if token in dictionary.keys():
      cands = cands & set(dictionary[token].docs) # intersection of doc lists where all words of a phrase are present
    else:
      return {}
  return cands

phrase = 'study'
candidates = set(list(range(1, len(dictionary)+1)))
print(len(candidates))

8996


In [13]:
def process(phrase:str)->List:
  res_list = []
  candidate_docs = candidates

  tokens = preprocess_query(phrase) #preprocess the query
  if len(tokens) == 0:
    return []
  candidate_docs = intersect_docs(phrase, dictionary, candidate_docs) #find out the candidate doc_ids as the intersection of all word postings within a phrase
  
  if len(candidate_docs) == 0:
    return []
  for doc_id in candidate_docs: #check each doc_id

    inter_list = dictionary[tokens[0]].docs[doc_id].positions # go to word -> doc_id -> postions list

    for idx in range(1, len(tokens)):

      pos_list = dictionary[tokens[idx]].docs[doc_id].positions #get the positions of the current word
      inter_list_copy = set([x+1 for x in inter_list]) 
      inter_list =  list(inter_list_copy & set(pos_list)) #check for intersection betweein phrase[0:i] and phrase[i]

    if len(inter_list) == 0: #if no intersection found do not add
      continue
    else:
      res_list.append(doc_id)

  return sorted(res_list)

In [14]:
#Mapping command to function

def solve(list1, list2):
    return AND(list1,list2)

In [15]:
source_file='/content/drive/MyDrive/IR_Assignments/A1/BIGRAM.pkl'
with open(source_file, 'rb') as f:
    BIGRAM=pickle.load(f)
doc_mapping  = {}
with open('/content/drive/MyDrive/IR_Assignments/A1/doc_id_mapping.pkl', 'rb') as f:
  doc_mapping = pickle.load(f)

In [16]:
#Solving generalised query

def compute(words, commands):
  sol=[]
  total=0
  query=''
  t1=words[0]
  t2=words[1]

  if(t1 not in BIGRAM):
    list1=[]
  else:
    list1=BIGRAM[t1]
  
  if(t2 not in BIGRAM):
    list2=[]
  else:
    list2=BIGRAM[t2]
  
  sol,comp=solve(list1,list2)
  query+=t1+' '+commands[0]+' '+t2

      
  total+=comp

  for i in range(2,len(words)):

    t=words[i]

    if(t not in BIGRAM):
      lst=[]
    else:
      lst=BIGRAM[t]
    
    sol,comp=solve(sol,lst,commands[i-1])
    
    total+=comp
    query+=' '+commands[i-1]+' '+t

  return query, sol, total  

In [17]:
def AND(list1, list2):
  comp=0

  if list1==[] or list2==[]:
    return [],comp

  sol=[]

  idx1=0
  idx2=0

  while idx1<len(list1)  and idx2< len(list2):
    comp+=1

    if list1[idx1]==list2[idx2]:
      sol.append(list1[idx1])
      idx1+=1
      idx2+=1
    
    elif list1[idx1]<list2[idx2]:
      idx1+=1
    
    else:
      idx2+=1

  return sol,comp


In [18]:
#Solving preprocessed query

def simpler_query(tokens):

  words_lst=tokens


  words=[words_lst[i]+' '+words_lst[i+1] for i in range(len(words_lst)-1) if (words_lst[i].strip()!='' and words_lst[i+1].strip()!='')]
  # print(words)

  commands=['AND' for i in range(len(words)-1)]

  return compute(words,commands)

In [19]:
#Function to process query

def query(no, inp):

  tokens=preprocess_query(inp)

  if(len(tokens)==0):
     print('Number of documents retrieved for query ' +str(no)+' using Bigram Inverted Index : 0')
     print('Names of the documents retrieved for query '+ str(no)+ ' using Bigram Inverted Index : None')
     return


  query, ans, comparisions= simpler_query(tokens)
  docNames=''
  for id in ans:
    docNames+=doc_mapping[id]+' '

  
  print('Number of documents retrieved for query ' +str(no)+' using Bigram Inverted Index : '+ str(len(ans)))
  print('Names of the documents retrieved for query '+ str(no)+ ' using Bigram Inverted Index : '+ docNames)



In [21]:
def run_query():
  t = int(input("Enter the number of queries: "))
  idx = 1
  while t > 0:
    phrase = input("Enter the phrase: ")
    res_list_using_pi = process(phrase) #Positional
    res_list_using_bi = query(idx, phrase) #Bigram

    print(f'Number of documents retrieved for query {idx} using positional index: {len(res_list_using_pi)}')
    print(f'Names of documents retrieved for query {idx} using positional index: ', end = " ")
    for doc_id in res_list_using_pi:
      print(doc_id_mapping[doc_id], end=", ")
    print()
    idx+=1
    t-=1
run_query()

Enter the number of queries: 2
Enter the phrase: this is in
Number of documents retrieved for query 1 using Bigram Inverted Index : 0
Names of the documents retrieved for query 1 using Bigram Inverted Index : None
Number of documents retrieved for query 1 using positional index: 0
Names of documents retrieved for query 1 using positional index:  
Enter the phrase: in a the
Number of documents retrieved for query 2 using Bigram Inverted Index : 0
Names of the documents retrieved for query 2 using Bigram Inverted Index : None
Number of documents retrieved for query 2 using positional index: 0
Names of documents retrieved for query 2 using positional index:  
